# DTI Deterministic Tractography

Local fibre tracking is an approach used to model white matter fibres by creating streamlines from local directional information. The idea is as follows: if the local directionality of a tissue segment is known, one can integrate along those directions to build a complete representation of the structure.

To perform local fibre tracking we needed:

1) A method for estimating directions from a diffusion data set.

2) A set of seeds from which to begin propagating streamlines

3) A set of stopping criteria for streamline propagation

4) A method for propagating streamlines.

This notebook will perform a tractography reconstruction from a dMRI dataset using the DTI model.

Load the Stanford's dMRI dataset:

In [3]:
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti

hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

The loaded data is masked:

In [4]:
from dipy.segment.mask import median_otsu

maskdata, mask = median_otsu(data, vol_idx=range(0, 9),
                             numpass=1, dilate=5)

We are provided a file that labels white matter voxels of the Stanford's dMRI dataset by either 1 or 2. To speed the processing  we'll use these labels to only extract peaks in the labeled white matter voxels: 

In [5]:
from dipy.io.image import load_nifti_data

label_fname = get_fnames('stanford_labels')

labels = load_nifti_data(label_fname)

white_matter = (labels == 1) | (labels == 2)

### 1) Getting directions from the dMRI dataset

We will use the DTI model to get the directions. For this we instantiate a Tensor model according to the data GradientTable object:

In [6]:
from dipy.reconst.dti import TensorModel

dti_model = TensorModel(gtab)

Diffusion tensor main direction can be calculated from its eigenvectors. Here, DTI peaks are extracted using dipy's function "peaks_from_model". This function provides a more general strategy to extract directions that can be applied to different dMRI models. Below we extract directions on the white matter voxels.

* Note: Function "peaks_from_model" requires a variable containing discrete directions for evaluation.

In [7]:
from dipy.direction import peaks_from_model
from dipy.data import default_sphere

dti_peaks = peaks_from_model(dti_model, maskdata, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=white_matter)

We will visualize a slice from the direction field which we will use as the basis to perform the tracking.

In [7]:
from dipy.viz import window, actor, has_fury

if has_fury:
    scene = window.Scene()
    scene.add(actor.peak_slicer(dti_peaks.peak_dirs,
                                dti_peaks.peak_values,
                                colors=None))

    window.record(scene, out_path='dti_direction_field.png', size=(900, 900))
    window.show(scene, size=(800, 800))

### 2) Defining the seeds where tracking will start

The seeds for tracking are defined in Dipy using the function "seeds_from_mask" from "dipy.tracking.utils". We place a grid of 2 × 2 × 2 grid of seeds per voxel, in a sagittal slice of the corpus callosum. The voxels to seed are selected by the input "seed_mask" which here we use the regions with the label value 2 which corresponds to voxels in a sagittal slice of the corpus callosum. Tracking from this region will, therefore, give us a model of the corpus callosum tract.

In [9]:
from dipy.tracking import utils

seed_mask = (labels == 2)
seeds = utils.seeds_from_mask(seed_mask, affine, density=[3, 3, 3])

### 3) Defining the tracking stopping criterion

Streamlines will be produced until reaching a voxel with FA lower than 0.2. For this, we have to first compute the FA map.

In [10]:
dti_fit = dti_model.fit(maskdata)
fa = dti_fit.fa

The stopping criterion area is defined using the function "ThresholdStoppingCriterion" from "dipy.tracking.stopping_criterion"

In [11]:
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

stopping_criterion = ThresholdStoppingCriterion(fa, .2)

### 4) Propagating the streamlines

Having the directions, starting position and stopping criterion defined we can start generating streamlines. The streamline generation has to be first initialized. For this, we use the function "LocalTracking" from "dipy.tracking.local_tracking"

In [12]:
from dipy.tracking.local_tracking import LocalTracking

streamlines_generator = LocalTracking(dti_peaks, stopping_criterion, seeds,
                                      affine=affine, step_size=.5)

Streamlines can then be generated using the "Streamline" function from dipy.tracking.streamline

In [13]:
from dipy.tracking.streamline import Streamlines

# Generate streamlines object
streamlines = Streamlines(streamlines_generator)

Below we display the resulting streamlines using the fury python package.

In [1]:
from dipy.viz import colormap

if has_fury:
    # Prepare the display objects.
    color = colormap.line_colors(streamlines)

    streamlines_actor = actor.line(streamlines,
                                   colormap.line_colors(streamlines))

    # Create the 3D display.
    scene = window.Scene()
    scene.add(streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    window.record(scene, out_path='dti_tractogram.png', size=(800, 800))
    window.show(scene)

NameError: name 'has_fury' is not defined

We’ve created a deterministic set of streamlines using the LocalTracking algorithm from the diffusion tensor main direction. This procedure is called deterministic because if you repeat the fiber tracking (keeping all the inputs the same) you will get exactly the same set of streamlines. We can save the streamlines as a Trackvis file so it can be loaded into other software for visualization or further analysis.

In [14]:
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk

sft = StatefulTractogram(streamlines, hardi_img, Space.RASMM)
save_trk(sft, "dti_CC_tractography_deterministic.trk", streamlines)

In [6]:
!dir

 Volume in drive D is Data
 Volume Serial Number is 6A19-EF97

 Directory of D:\Projects\EPICS\Notebooks

05/20/2023  05:18 PM    <DIR>          .
05/20/2023  05:18 PM    <DIR>          ..
05/16/2023  10:20 AM    <DIR>          .ipynb_checkpoints
05/16/2023  01:32 AM         9,889,472 Bilder_DTI_spline_filter.trk
05/14/2023  04:04 PM        18,007,672 dti_CC_tractography_deterministic.trk
05/13/2023  04:23 PM            33,642 dti_direction_field.png
05/13/2023  04:25 PM           251,579 dti_tractogram.png
05/16/2023  01:51 AM         6,733,714 ICBM_DTI-81_Atlas_Fractional_Anisotropy_raw.zip
07/29/2013  10:16 AM         6,760,247 ICBM_FA_raw.zip
11/05/2009  10:46 AM    <DIR>          ICBM_FA_raw.zipfile
03/10/2023  11:30 PM            41,568 multi_tensor_voxel_fit.png
03/10/2023  11:30 PM            40,261 multi_tensor_voxel_gt.png
03/10/2023  11:31 PM            60,401 odf_multi_tensor.png
03/10/2023  11:29 PM            49,452 odf_single_tensor.png
03/10/2023  11:28 PM            28

In [2]:
!dipy_horizon dti_CC*trk

Loading file ...
dti_CC_tractography_deterministic.trk




In [10]:
!dipy_horizon dti_CC*trk tensor_fa.nii.gz

Loading file ...
dti_CC_tractography_deterministic.trk


Loading file ...
tensor_fa.nii.gz


Affine to RAS
[[   2.    0.    0.  -80.]
 [   0.    2.    0. -120.]
 [   0.    0.    2.  -60.]
 [   0.    0.    0.    1.]]
Original shape (81, 106, 76)
Resized to RAS shape  (81, 106, 76)


In [21]:
# from dipy.tracking import streamline, metrics
# from dipy.tracking import distances
# from dipy.reconst.dti import mean_diffusivity

# lengths = streamline.length(streamlines)
# md_values = mean_diffusivity(data, streamlines)
# fa_values = metrics.mean_fa(streamlines, data)
# rd_values = metrics.mean_rd(streamlines, data)
# ad_values = metrics.mean_ad(streamlines, data)

In [8]:
!dipy_horizon dti_CC*trk tensor_md.nii.gz

Loading file ...
dti_CC_tractography_deterministic.trk


Loading file ...
tensor_md.nii.gz


Affine to RAS
[[   2.    0.    0.  -80.]
 [   0.    2.    0. -120.]
 [   0.    0.    2.  -60.]
 [   0.    0.    0.    1.]]
Original shape (81, 106, 76)
Resized to RAS shape  (81, 106, 76)


In [7]:
!dipy_horizon dti_CC*trk tensor_ad.nii.gz

Loading file ...
dti_CC_tractography_deterministic.trk


Loading file ...
tensor_ad.nii.gz


Affine to RAS
[[   2.    0.    0.  -80.]
 [   0.    2.    0. -120.]
 [   0.    0.    2.  -60.]
 [   0.    0.    0.    1.]]
Original shape (81, 106, 76)
Resized to RAS shape  (81, 106, 76)


In [9]:
!dipy_horizon dti_CC*trk tensor_rd.nii.gz

Loading file ...
dti_CC_tractography_deterministic.trk


Loading file ...
tensor_rd.nii.gz


Affine to RAS
[[   2.    0.    0.  -80.]
 [   0.    2.    0. -120.]
 [   0.    0.    2.  -60.]
 [   0.    0.    0.    1.]]
Original shape (81, 106, 76)
Resized to RAS shape  (81, 106, 76)
